In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\Project\\e2e\\ml_practice'

In [5]:
os.environ['MLFLOW_TRACKING_URI'] = "https://dagshub.com/dhanushnayak/ml_practice.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "dhanushnayak"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "253df65190e7dc9ee74592f4802bc08e1df73a7e"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.comman import read_yaml, create_directories, save_bin, save_json

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath  = CONFIG_FILE_PATH, params_filepath =  PARAMS_FILE_PATH, schema_filepath =  SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params =  read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config =  ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/dhanushnayak/ml_practice.mlflow"
        )

        return model_evaluation_config

In [9]:
config =  ConfigurationManager()
config.get_model_evaluation_config()

[2023-07-30 16:56:05,362: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-07-30 16:56:05,368: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-07-30 16:56:05,386: INFO: comman: yaml file: schema.yaml loaded successfully]
[2023-07-30 16:56:05,390: INFO: comman: created directory at: artifacts]
[2023-07-30 16:56:05,394: INFO: comman: created directory at: artifacts/model_evaluation]


ModelEvaluationConfig(root_dir='artifacts/model_evaluation', test_data='artifacts/data_transformation/test.csv', model_path='artifacts/model_trainer/model.joblib', all_params=ConfigBox({'alpha': 0.5, 'l1_ratio': 0.7}), metric_file_name='artifacts/model_evaluation/metrics.json', target_column='quality', mlflow_uri='https://dagshub.com/dhanushnayak/ml_practice.mlflow')

In [12]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [17]:
class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return {"rmse":rmse,"mae":mae,"r2":r2}
    
    def log_into_mlfow(self):

        test_data = pd.read_csv(self.config.test_data)
        model = joblib.load(self.config.model_path)

        test_x =  test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store =  urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store,mlflow.get_tracking_uri(),urlparse(mlflow.get_tracking_uri()))

        with mlflow.start_run():
            predictions = model.predict(test_x)

            scores =  self.eval_metrics(test_y,predictions)

            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)

            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model,"model",registered_model_name='ElasticModel')
            else:
                mlflow.sklearn.log_model(model,'model')



In [18]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.log_into_mlfow()
except Exception as e:
    raise e

[2023-07-30 17:47:28,780: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-07-30 17:47:28,784: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-07-30 17:47:28,792: INFO: comman: yaml file: schema.yaml loaded successfully]
[2023-07-30 17:47:28,795: INFO: comman: created directory at: artifacts]
[2023-07-30 17:47:28,798: INFO: comman: created directory at: artifacts/model_evaluation]
https https://dagshub.com/dhanushnayak/ml_practice.mlflow ParseResult(scheme='https', netloc='dagshub.com', path='/dhanushnayak/ml_practice.mlflow', params='', query='', fragment='')
[2023-07-30 17:47:29,434: INFO: comman: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticModel' already exists. Creating a new version of this model...
2023/07/30 17:47:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticModel, version 2
Created version '2' of model 'ElasticModel'.
